**Title : Product attribute Extraction**



Importing Libraries

In [ ]:
#tensorflow_version 2.x
!pip install tensorflow
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.initializers import Constant


Loading the dataset

In [ ]:
dataset = pd.read_csv('train.csv')

Visualizinig the dataset

In [ ]:
dataset.head()

Taking the important features of the dataset for further processing

In [ ]:
dataset= dataset[['title','brand']]

We will do the tokenization of the words persent in the dataset. Tokenizer will split the data which will then be used for mapping as a numeric vector by 
the help of glove.

In [ ]:
tokenlist = []
tokenlist.append(dataset['title'])
tokenlist.append(dataset['brand'])
tokenlist = np.ravel(tokenlist)
print(tokenlist)

In [ ]:
featurestoCare = 20000
tokens = Tokenizer(num_words=featurestoCare)
tokens.fit_on_texts(tokenlist)
x = tokens.texts_to_sequences(dataset['title'].values)
x = pad_sequences(x)


In [ ]:
y = dataset['brand'].values
y = tokens.texts_to_sequences(y)
y = pad_sequences(y)

Train-Test Data Splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1,random_state=44)

Word Embedding

In [ ]:
embeddings_index = {}

# Glove Embedding Link: https://nlp.stanford.edu/projects/glove/
f = open('glove.6B.300d.txt',encoding="utf8") #Glove Embedding File 
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
list(word_index.values())[-1]

In [ ]:
word_index = tokens.word_index
print('Found %s unique tokens.' % list(word_index.values())[-1])

In [ ]:
embedding_dim = 300
num_words = min(featurestoCare, list(word_index.values())[-1]) + 1
# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))
# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    elif embedding_vector is None:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)



In [ ]:
T = X_train.shape[1]

Bi-LSTM Model

In [ ]:
model = Sequential()
# Embedding Layer 
model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=T,
                    trainable=True))

# Bi_LSTM Layer
model.add(Bidirectional(LSTM(100,return_sequences=True, go_backwards= True)))
model.add((LSTM(100, go_backwards= True)))

#Dense Layer
model.add(Dense(5, activation = 'relu'))


#Model Compilation
model.compile(loss = 'hinge', optimizer='Adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
#Trainining Data
print(X_train)

In [ ]:
y_train.shape

Model Fitting

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Prediction in Vector

In [ ]:
ypre = model.predict(X_test)

In [ ]:
print(ypre)

In [ ]:
len = ypre.shape[0]

Converting the prediction from float to integer

In [ ]:
i=0
while i< len:
    ypre[i][4] = round(ypre[i][4])
    ypre[i][3] = round(ypre[i][3])
    ypre[i][2] = round(ypre[i][2])
    ypre[i][1] = round(ypre[i][1])
    ypre[i][0] = round(ypre[i][0])
    i = i+1

Vector to Word

In [ ]:
predicted = tokens.sequences_to_texts(ypre)

Prediction in Words

In [ ]:
print(predicted)